# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
ctab=data.groupby('race')['call'].agg(['size','sum'])
ctab

,size,sum
race,,
b,2435,157.0
w,2435,235.0


<div class="span5 alert alert-success">
    <p>Q1: We can perform a z-score test for the difference between two population proportions to evaluate whether race has a significant impact on the number of callbacks for resumes.  Another approach would be a chi square test to evaluate whether there was a relationship between race and the proportion of callbacks. </p>
    <p>The CLT applies because we need a large enough sample for the sampling distributions of callback proportion to be represented with a normal distribution.  The requirement is np > 10 and n(1-p)>10, which the dataset appears to meet for both B and W samples.</p>
    <p> Q2: The null hypothesis of either test is that proportion of resumes getting callbacks is the same for either race.  The alternate hypothesis is that the callback rate <i>is</i> affected by race.</p>

</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']


In [7]:
# Your solution to Q3 here

# What is the likelihood of observing a difference in population proportion as large as observed under Ho?

# Frequentist approach: Apply hypothesis test for difference in proportions 
# Compute margin of error, confidence interval, and p-value. 

#sample counts
n_w = w['call'].size
n_b = b['call'].size

#observed sample proportions
prop_w=w['call'].sum()/n_w
prop_b=b['call'].sum()/n_b

#observed difference in sample proportions
prop_del = np.abs(prop_b-prop_w)

# standard deviation of difference in sample proportions
sd_del = np.sqrt((prop_w*(1-prop_w))/n_w + (prop_b*(1-prop_b))/n_b)

# 95% Margin of error and confidence interval for the difference in proportions

zc = stats.norm.ppf(0.025) # z-score for 95% CI
MOE_del = np.array([zc*sd_del,(0-zc)*sd_del])
CI_del = prop_del+MOE_del

# p-value (for test that difference in proportions = 0)
#combined proportion
z = prop_del/sd_del # z-score for observed difference in proportions relative to null hypothesis
pHo = 1-stats.norm.cdf(z) # p-value 

print("Margin of Error = {}\nConfidence Interval = {}\nP-value = {}".format(MOE_del,CI_del,pHo))
print("We reject the null hypothesis and conclude the proportion of callbacks differs between races.")

Margin of Error = [-0.01525513  0.01525513]
Confidence Interval = [0.01677773 0.04728798]
P-value = 1.9312826037620745e-05
We reject the null hypothesis and conclude the proportion of callbacks differs.


In [20]:
#Another approach:  Chi-square test for independence of race category
#Ho = callback proportion is not dependent on race
conting = np.array([[n_w*prop_w,n_w*(1-prop_w)],[n_b*prop_b,n_b*(1-prop_b)]])
chi2,p,dof,exv=stats.chi2_contingency(conting)
print('P-value for chi-square test: '+str(p)+'\nWe conclude that callback proportion is dependent on race.')

4.997578389963255e-05


In [8]:
# Bootstrap approach:  Run permutation test where the b/w assignments are scrambled and randomly assigned.  
# What is likelihood of getting a difference in proportions as high as was observed in the original dataset?

# Run 10,000 permutations of the combined calls results, split into "B" and "W" portions, calculate absolute difference in proportions
rep_deltas=np.empty(10000)
callbacks = data['call'].values

for i in range(10000):
    callbacks_scrambled = np.random.permutation(callbacks) #scramble the callbacks
    w_call=callbacks_scrambled[0:n_w] # Half goes to "W"
    b_call=callbacks_scrambled[n_w:] # Half goes to "B"
    w_prop=np.sum(w_call)/n_w 
    b_prop=np.sum(b_call)/n_b
    rep_deltas[i]=np.abs(b_prop-w_prop) #Absolute value of difference between "B" and "W" in permuted dataset

extremes = sum(rep_deltas>prop_del)/10000 #Compare permutations to observed difference
print('Fraction of permutation replicates with differences in callback proportion at least as extreme as observed: '+str(extremes))
print('Again we reject Ho and conclude the proportion of callbacks differs')

Fraction of permutation replicates with differences in callback proportion at least as extreme as observed: 0.0001
Again we reject Ho and conclude the proportion of callbacks differs


<div class="span5 alert alert-success">
<p> For this assignment, we were presented with a dataset of records corresponding to individual resumes.  This dataset was derived from a research study designed to infer whether there was evidence of racial bias in the rate of callbacks from employers when reviewing resumes.  The resumes were randomly assigned black and white-sounding names, and the presence of a callback was recorded in a 'call' field, with a value of 1 indicating the applicant received a callback and 0 indicating no callback occurred. </p>
   <p>Callback rates of 6.4% and 9.7% were observed for the black and white resumes, respectively.  To evaluate whether the difference between these rates was statistically significant, we performed a hypothesis test for difference in proportion of callbacks between races using both frequentist (i.e. z-score) and bootstrapping techniques.  Both methods produced very low p-values, suggesting that the observed difference in callback rate was not a chance occurrence, but rather evidence of a real difference in callback rates between the white and black resume groups.  The results of a chi-square test also verified that the callback rates were dependent on the race assigned to a resume in this study.
</p>
    <p>Although these results demonstrate that applicant race affects the likelihood of getting a callback, this does not mean it is the only factor that influences an employer's decision. We assume that the original investigators controlled for other resume items (such as skills, years of experience, type of education, etc.) to prevent these factors from biasing the experiment.  If these factors were included in our analysis, we could evaluate the relative influence of race in the context of these other elements to infer the most important factors influencing employer response.  </p>
</div>